In [128]:
import os
import geopandas as gpd
import pandas as pd

In [129]:
## 서울시 H3 그리드 불러오기
seoul_grid = gpd.read_file("data/그리드/seoul_grid.shp")
seoul_grid.head(3)

,h3_9,geometry
0,8930e0a5423ffff,"POLYGON ((126.79014 37.60392, 126.78916 37.602..."
1,8930e1d8127ffff,"POLYGON ((126.97254 37.56289, 126.97156 37.561..."
2,8930e1c24afffff,"POLYGON ((127.03110 37.66834, 127.03012 37.667..."


In [130]:
seoul_grid.shape

(23024, 2)

In [131]:
path = 'data/토지데이터/NGII_LUM_11_서울/'
file_list = os.listdir(path)
file_list = [file for file in file_list if file.endswith('.shp')]

In [132]:
land = gpd.GeoDataFrame()
for i in file_list:
    data = gpd.read_file(path + i, crs='5181')
    land = pd.concat([land,data])
    
land = land.reset_index(drop = True)

In [133]:
land_info = pd.read_excel('data/토지이용현황도_분류항목(코드).xls', header = 5)
land_info.head(3)

,대분류,중분류,소분류,코드,정의(38개),Unnamed: 5
0,농지,논,경지정리답,1110,물을 이용하여 벼를 재배하기 위한 토지로 수로가 용수를 공급하는 경지 정리지,NaN
1,NaN,NaN,미경지정리답,1120,"물을 이용하여 벼를 재배하기 위한 토지로서 주로 계단 형태이며, 경사가 심한 산간지...",NaN
2,NaN,밭,"보통,특수작물",1210,"물을 대지 아니하고 식물을 재배하는 토지로 보통작물인 무, 배추, 시금치 등의 채소...",NaN


In [134]:
land_info['대분류'][:4] = "농지"; land_info['대분류'][4:13] = "임지"
land_info['대분류'][13:32] = "도시 및 주거지"; land_info['대분류'][32:] = "수계"

C:\Users\user\AppData\Local\Temp\ipykernel_1524\4170629735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land_info['대분류'][:4] = "농지"; land_info['대분류'][4:13] = "임지"
C:\Users\user\AppData\Local\Temp\ipykernel_1524\4170629735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land_info['대분류'][13:32] = "도시 및 주거지"; land_info['대분류'][32:] = "수계"


In [135]:
land_info = land_info[['대분류', '코드']]
land_info.head(3)

,대분류,코드
0,농지,1110
1,농지,1120
2,농지,1210


In [136]:
land_info = land_info.astype({'코드' : 'str'}) # merge 위해 자료형 맞춰주기

In [137]:
land = pd.merge(land, land_info, how = "inner", left_on = "UCB", right_on = "코드")
land.head(3)

,UCB,geometry,대분류,코드
0,2210,"POLYGON ((188722.870 458389.790, 188711.470 45...",임지,2210
1,2210,"POLYGON ((188718.060 459947.730, 188727.810 45...",임지,2210
2,2210,"POLYGON ((188714.520 460626.560, 188733.040 46...",임지,2210


In [138]:
land = gpd.GeoDataFrame(land, geometry = 'geometry', crs = '5181')
land = land.to_crs(4326)

In [139]:
seoul_cen = gpd.GeoDataFrame(seoul_grid, geometry = "geometry", crs = "4326")
seoul_cen.head(3)

,h3_9,geometry
0,8930e0a5423ffff,"POLYGON ((126.79014 37.60392, 126.78916 37.602..."
1,8930e1d8127ffff,"POLYGON ((126.97254 37.56289, 126.97156 37.561..."
2,8930e1c24afffff,"POLYGON ((127.03110 37.66834, 127.03012 37.667..."


In [140]:
seoul_cen['geometry'] = seoul_cen['geometry'].centroid
seoul_cen.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_1524\844776798.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  seoul_cen['geometry'] = seoul_cen['geometry'].centroid


,h3_9,geometry
0,8930e0a5423ffff,POINT (126.78841 37.60413)
1,8930e1d8127ffff,POINT (126.97081 37.56310)
2,8930e1c24afffff,POINT (127.02937 37.66856)


In [141]:
land_grid = gpd.sjoin(seoul_cen, land, how = "left")

In [142]:
land = land_grid[['h3_9', '대분류', '코드', 'geometry']]
land.head(3)

,h3_9,대분류,코드,geometry
0,8930e0a5423ffff,도시 및 주거지,3210,POINT (126.78841 37.60413)
1,8930e1d8127ffff,도시 및 주거지,3430,POINT (126.97081 37.56310)
2,8930e1c24afffff,임지,2210,POINT (127.02937 37.66856)


In [143]:
land1 = pd.DataFrame(land)
land1.drop('geometry', axis = 1, inplace = True)

In [144]:
from shapely.geometry import Polygon
import h3

In [145]:
## h3 index를 polygon으로 변환
def to_polygon(l) :
    return Polygon(h3.h3_to_geo_boundary(l, geo_json = True))

land1['geometry'] = land1['h3_9'].apply(to_polygon)
# land1 = land1.set_crs(epsg = 4326)

In [146]:
df = gpd.GeoDataFrame(land1, geometry = 'geometry', crs = '4326')

In [147]:
def polygon_to_coordinates(x) :
    lon, lat = x.exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

In [148]:
df['coordinates'] = df['geometry'].apply(polygon_to_coordinates)
del df['geometry']

In [149]:
df = pd.DataFrame(df)

In [150]:
df.head(3)

,h3_9,대분류,코드,coordinates
0,8930e0a5423ffff,도시 및 주거지,3210,"[[126.79013552025086, 37.60391518987627], [126..."
1,8930e1d8127ffff,도시 및 주거지,3430,"[[126.97254288416778, 37.56288514788764], [126..."
2,8930e1c24afffff,임지,2210,"[[127.03110097938165, 37.66834454055887], [127..."


In [151]:
df['color'] = df['코드'].str[:1]

In [152]:
df['color'].unique()

array(['3', '2', '1', nan, '4'], dtype=object)

- `대분류`, `코드` column의 결측값 채우기

In [153]:
df['코드'] = df['코드'].fillna(0)

In [154]:
df['color'] = df['color'].fillna(0)

In [155]:
df['대분류'] = df['대분류'].fillna('알수없음')

In [156]:
df['ID'] = df['대분류'] + '-' + df['코드'].astype('str')

In [157]:
import numpy as np

random_R = np.random.rand(34)
random_B = np.random.rand(34)
random_G = np.random.rand(34)

In [158]:
from tqdm import tqdm

In [159]:
data_list = []

for idx, row in tqdm(enumerate(df.groupby('color'))):
    sub = row[1]
    sub['color_R'] = random_R[idx]
    sub['color_B'] = random_B[idx]
    sub['color_G'] = random_G[idx]
    
    data_list.append(sub)

df  = pd.concat(data_list)
df = df.reset_index(drop=True)

5it [00:00, 499.89it/s]


In [186]:
df.loc[df["color"] == "1", "color_R"] = 0
df.loc[df["color"] == "1", "color_G"] = 255
df.loc[df["color"] == "1", "color_B"] = 0

In [187]:
df.loc[df["color"] == "2", "color_R"] = 255
df.loc[df["color"] == "2", "color_G"] = 255
df.loc[df["color"] == "2", "color_B"] = 0

In [188]:
df.loc[df["color"] == "3", "color_R"] = 255
df.loc[df["color"] == "3", "color_G"] = 0
df.loc[df["color"] == "3", "color_B"] = 255

In [189]:
df.loc[df["color"] == "4", "color_R"] = 0
df.loc[df["color"] == "4", "color_G"] = 0
df.loc[df["color"] == "4", "color_B"] = 255

In [194]:
d = df[df['color'] == 0].index
df.drop(d, inplace = True)

In [191]:
import pydeck as pdk

- 도시 및 주거지 : 선홍색
- 임지 : 노란색
- 농지 : 초록색
- 수계 : 파란색

In [195]:
layer = pdk.Layer(
    "PolygonLayer",
    df,
    get_polygon = "coordinates",
    get_fill_color = '[1 * color_R,  1 * color_G , 1 * color_B]',
    pickable = True,
    auto_highlight = True
)

# 서울시청 주소
lat = 37.5662; lng = 126.9787
seoul = [lat, lng]

view_state = pdk.ViewState(
    longitude = seoul[1],
    latitude = seoul[0],
    zoom = 14
)

r = pdk.Deck(layers = [layer], initial_view_state = view_state)
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…